In [ ]:
# !pip install transformers datasets torch
# !pip install transformers[torch]


## Import Libs

In [2]:
import os
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

import torch

c:\Users\cibel\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import Data

In [3]:
caminho_redacoes_corrigidas = os.path.join('..', 'data', 'redacoes_corrigidas_uol_fora_enem.csv')

df_redacoes_corrigidas = pd.read_csv(caminho_redacoes_corrigidas, sep=";")
df_redacoes_corrigidas.head()

,tema,description,title,text,comments,criteria_score_1,criteria_score_2,criteria_score_3,criteria_score_4,criteria_score_5
0,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,Natureza caótica,A natureza possui um equilíbrio singular singu...,"Texto muito bom, com alguns erros e equívocos ...",160,200,160,200,160
1,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,Em busca de um novo lar,"Com o aumento do aquecimento global, várias es...",Texto mediano. Além dos problemas pontuais neg...,120,120,120,120,120
2,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,A ação antrópica e suas consequências,"Há, na pré-história, pinturas rupestres que re...","Texto prolixo e repetitivo, com problemas pont...",120,120,80,120,120
3,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,O aquecimento global e os ursos,O aquecimento global vem afetando o mundo nos ...,"Lamentavelmente, o texto é fraco, insuficiente...",120,80,80,80,80
4,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,As consequências do degelo no Ártico,"O aumento do aquecimento global provocou, no í...",Texto mediano. O autor cumpre as exigências da...,80,120,120,120,120


In [4]:
df_redacoes_corrigidas["nota_final"] = df_redacoes_corrigidas["criteria_score_1"] + df_redacoes_corrigidas["criteria_score_2"] + df_redacoes_corrigidas["criteria_score_3"] + df_redacoes_corrigidas["criteria_score_4"] + df_redacoes_corrigidas["criteria_score_5"]

In [5]:
df_redacoes_corrigidas.head()

,tema,description,title,text,comments,criteria_score_1,criteria_score_2,criteria_score_3,criteria_score_4,criteria_score_5,nota_final
0,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,Natureza caótica,A natureza possui um equilíbrio singular singu...,"Texto muito bom, com alguns erros e equívocos ...",160,200,160,200,160,880
1,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,Em busca de um novo lar,"Com o aumento do aquecimento global, várias es...",Texto mediano. Além dos problemas pontuais neg...,120,120,120,120,120,600
2,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,A ação antrópica e suas consequências,"Há, na pré-história, pinturas rupestres que re...","Texto prolixo e repetitivo, com problemas pont...",120,120,80,120,120,560
3,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,O aquecimento global e os ursos,O aquecimento global vem afetando o mundo nos ...,"Lamentavelmente, o texto é fraco, insuficiente...",120,80,80,80,80,440
4,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,As consequências do degelo no Ártico,"O aumento do aquecimento global provocou, no í...",Texto mediano. O autor cumpre as exigências da...,80,120,120,120,120,560


In [4]:
df_redacoes_corrigidas['tema'][0]

'Os ursos polares da Rússia e um dilema ecológico'

In [5]:

# Passo 2: Pré-processar os dados
# Unir a redação e correção no formato de entrada do modelo

prompt_input = """
    Você é um estudante do ensino médio que está pretando vestibular para o ENEM, o Exame Nacional do Ensino Médio. 
    Você deve se atentar à 
     - Domínio da norma-padrão da língua escrita; 
     - Compreensão do tema; 
     - Organização das informações e argumentos;
     - Correta aplicação da lógica;
    - Apresentar uma proposta de intervenção para o problema.
    
    O tema da redaçãao neste ano é "{}". 
    E você escreveu a seguinte redação:

    {}
""".format(df_redacoes_corrigidas['tema'][0],  df_redacoes_corrigidas['text'][0])

print(prompt_input)

In [7]:
prompt_ouput = """
    Você é um estudante do ensino médio que está pretando vestibular para o ENEM, o Exame Nacional do Ensino Médio. 
    Você deve se atentar à 
     - Domínio da norma-padrão da língua escrita; 
     - Compreensão do tema; 
     - Organização das informações e argumentos;
     - Correta aplicação da lógica;
    - Apresentar uma proposta de intervenção para o problema.
    
    O tema da redaçãao neste ano é "{}". 
    E você escreveu a seguinte redação:

    {}
""".format(df_redacoes_corrigidas['tema'][0],  df_redacoes_corrigidas['text'][0])

print(prompt_input)


    Você é um estudante do ensino médio que está pretando vestibular para o ENEM, o Exame Nacional do Ensino Médio. 
    Você deve se atentar à 
     - Domínio da norma-padrão da língua escrita; 
     - Compreensão do tema; 
     - Organização das informações e argumentos;
     - Correta aplicação da lógica;
    - Apresentar uma proposta de intervenção para o problema.
    
    O tema da redaçãao neste ano é "Os ursos polares da Rússia e um dilema ecológico". 
    E você escreveu a seguinte redação:

    A natureza possui um equilíbrio singular singular, de modo que todos os seres vivos desempenham uma função - os ursos polares, por exemplo, ao se alimentarem de peixes e focas focas, regulam a quantidade de indivíduos nessas populações. Contudo, os homens ainda não compreendem a importância do respeito ao meio ambiente para a preservação da própria espécie e, por isso, têm arcado com as consequências, como o ataque dos ursos polares a comunidades humanas por diminuição da oferta de al

In [ ]:

df_redacoes_corrigidas['input'] = "Tema: " + df_redacoes_corrigidas['tema'] + "\n\nRedação:\n" + df_redacoes_corrigidas['text'] + "\n\nCorreção:\n" + df_redacoes_corrigidas['comments']
df_redacoes_corrigidas['output'] = "A corre~ção da sua Correção:\n" + df_redacoes_corrigidas['comments']

# Passo 3: Criar um Dataset para o Hugging Face
train_data = Dataset.from_pandas(df_redacoes_corrigidas[['input', 'output']])


In [14]:
max_length = 750

In [15]:

# Passo 4: Tokenização das entradas e saídas
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Definir o token de pad

def tokenize_function(examples):
    inputs = tokenizer(examples['input'], padding="max_length", truncation=True, max_length=max_length)
    outputs = tokenizer(examples['output'], padding="max_length", truncation=True, max_length=max_length)
    inputs['labels'] = outputs['input_ids']
    return inputs

tokenized_dataset = train_data.map(tokenize_function, batched=True)


Map: 100%|██████████| 3726/3726 [00:12<00:00, 307.78 examples/s]


In [16]:

# Passo 5: Configuração dos parâmetros de treinamento
training_args = TrainingArguments(
    output_dir="./results_enem",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Ajuste conforme necessário
    per_device_train_batch_size=4,  # Tamanho do lote, ajuste conforme necessário
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs_enem",
    logging_steps=10,
)

# Passo 6: Definir o Trainer
model = GPT2LMHeadModel.from_pretrained("gpt2")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [17]:
# model_name = "gpt2"
# model = GPT2LMHeadModel.from_pretrained(model_name)
print( model.config)

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 50257
}



In [18]:

# Iniciar o treinamento
trainer.train()

  0%|          | 10/2796 [02:55<11:25:32, 14.76s/it]

{'loss': 4.4867, 'grad_norm': 41.3036994934082, 'learning_rate': 4.982117310443491e-05, 'epoch': 0.01}


  1%|          | 20/2796 [05:19<10:58:30, 14.23s/it]

{'loss': 2.3286, 'grad_norm': 18.525035858154297, 'learning_rate': 4.964234620886982e-05, 'epoch': 0.02}


  1%|          | 30/2796 [07:43<11:02:12, 14.36s/it]

...

100%|█████████▉| 2790/2796 [11:07:25<01:25, 14.18s/it]

{'loss': 0.004, 'grad_norm': 0.25362375378608704, 'learning_rate': 1.072961373390558e-07, 'epoch': 2.99}


100%|██████████| 2796/2796 [11:08:47<00:00, 14.35s/it]

{'train_runtime': 40127.088, 'train_samples_per_second': 0.279, 'train_steps_per_second': 0.07, 'train_loss': 0.36265019896121775, 'epoch': 3.0}


TrainOutput(global_step=2796, training_loss=0.36265019896121775, metrics={'train_runtime': 40127.088, 'train_samples_per_second': 0.279, 'train_steps_per_second': 0.07, 'total_flos': 5841444667392000.0, 'train_loss': 0.36265019896121775, 'epoch': 3.0})

In [20]:
# Passo 8: Salvar o modelo e o tokenizer
model.save_pretrained("./modelo_finetuned_enem_v2")
tokenizer.save_pretrained("./modelo_finetuned_enem_v2")

('./modelo_finetuned_enem_v2\\tokenizer_config.json',
 './modelo_finetuned_enem_v2\\special_tokens_map.json',
 './modelo_finetuned_enem_v2\\vocab.json',
 './modelo_finetuned_enem_v2\\merges.txt',
 './modelo_finetuned_enem_v2\\added_tokens.json')

In [25]:
# Carregar o modelo e o tokenizer ajustados
model = GPT2LMHeadModel.from_pretrained("./modelo_finetuned_enem_v2")
tokenizer = GPT2Tokenizer.from_pretrained("./modelo_finetuned_enem_v2")

# Definir o dispositivo (GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# (lm_head): Linear(in_features=768, out_features=50257, bias=False) o out seria o= 1

In [26]:
def avaliar_redacao(redacao, tema):
    # Criar o prompt com a redação e o tema
    prompt = f"Tema: {tema}\n\nRedação:\n{redacao}\n\nCorreção e Notas:"
    
    # Tokenizar o prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    #inputs = tokenizer(prompt, return_tensors="pt").to(device) - # PT é de temnsorflow = TODO: Usar modelos, maritaca, sabias, caramelo
    
    # Gerar a saída do modelo
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1, do_sample=True, top_p=0.95, top_k=50)
    
    # Decodificar a resposta do modelo
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return resposta

In [23]:
# Exemplo de uso
redacao_exemplo = """
A redação do ENEM exige que os candidatos demonstrem domínio da norma culta da língua portuguesa, bem como capacidade de argumentação. Além disso, é necessário que os alunos apresentem uma proposta de intervenção para o problema discutido. No entanto, muitos estudantes acabam fugindo do tema ou apresentando argumentos fracos, o que compromete suas notas.
"""
tema_exemplo = "A importância da educação no Brasil"

correcao = avaliar_redacao(redacao_exemplo, tema_exemplo)
print(correcao)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Tema: A importância da educação no Brasil

Redação:

A redação do ENEM exige que os candidatos demonstrem domínio da norma culta da língua portuguesa, bem como capacidade de argumentação. Além disso, é necessário que os alunos apresentem uma proposta de intervenção para o problema discutido. No entanto, muitos estudantes acabam fugindo do tema ou apresentando argumentos fracos, o que compromete suas notas.


Correção e Notas:
4       120
       ... 
3721    160
3722    160
372323   160
3724    160
372524   160
Name: criteria_score,2, Length: 372626262626646464266464646464646464   3: 0       160
2        80
3        80
4       120
       ... 
3721    160
3722    160
3723    160
372423   160
3724    120
Name  criteria_score_3, Length: 3726 37 37 37 3726262626262626262626262626 37 37, 0       200
1       120
3        80
4       120
       ... 
3721    200
3722    200
3723    200
3724    200
3725    120
Name: criteria_score_4, Length: 3726 37 37 37: int64, Nota 5: 0       160
1       120
2

In [27]:
# Exemplo de uso
redacao_exemplo = """
A imigração de refugiado tem sido um dos maiores desafios das nações. Este transpassa a questão humanitária e vai de encontro a  soberania de muitos países. Não diferente, O  Brasil enfrenta, como os demais  situação parecida. Nos últimos anos uma grande quantidade de refugiados têm transpassada  a fronteira do país forçando o poder público a tomar medidas que garantam a integridade da soberania nacional e que esteja em consonância aos direitos humanos.  Várias nações do mundo estão passando por percalços, seja por  guerras, crises econômicas, conflitos internos, regimes ditatoriais... Fato que  faz surgir refugiados que vislumbram nos países mais próximos a salva guarda  para manterem a integridade. O que para estes  é um alento, para o país em que vão se refugiam  se torna um problema. Que muitas vezes vezes,  não está relacionado diretamente a um sentimento xenófobo, mas sim por  uma questão social. No caso do Brasil, os refugiados da nossa nação adjacente se abrigaram em uma cidade de um estado da federação  que tem grande dificuldade para cuidar até mesmo dos nativos. Causando  um verdadeiro caos. O que  fez aumentar ainda mais o sofrimento de quem já está em uma situação extrema. Destarte que  não basta o mero acolhimento, deve-se socorrer e fomentar uma nova expectativa de vida nestas  pessoas que estão distantes do país de origem. O que  tende a demandar tempo e recursos. No caso brasileiro  um dispêndio a mais em um país que está em busca de reduzir despesas. Salientando que se faz necessário uma previa  triagem para evitar que pessoas criminosas e mal-intencionadas venham fazer da nação um local para praticas criminosa  e potencialize a violência. Desprende  que o maior desafio no trato com os imigrantes está em conciliar a soberana força nacional de proteção com a aplicação de direitos humanos. É impensável deixar de ajudar o semelhante em apuros. Efetivar esse apoio sem prejudicar os demais é tarefa que requer recursos e planejamentos. O que aos poucos o país vai se consolidando.

"""
tema_exemplo = "Desafios na imigração de refugiados"

correcao = avaliar_redacao(redacao_exemplo, tema_exemplo)
print(correcao)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Tema: Desafios na imigração de refugiados

Redação:

A imigração de refugiado tem sido um dos maiores desafios das nações. Este transpassa a questão humanitária e vai de encontro a  soberania de muitos países. Não diferente, O  Brasil enfrenta, como os demais  situação parecida. Nos últimos anos uma grande quantidade de refugiados têm transpassada  a fronteira do país forçando o poder público a tomar medidas que garantam a integridade da soberania nacional e que esteja em consonância aos direitos humanos.  Várias nações do mundo estão passando por percalços, seja por  guerras, crises econômicas, conflitos internos, regimes ditatoriais... Fato que  faz surgir refugiados que vislumbram nos países mais próximos a salva guarda  para manterem a integridade. O que para estes  é um alento, para o país em que vão se refugiam  se torna um problema. Que muitas vezes vezes,  não está relacionado diretamente a um sentimento xenófobo, mas sim por  uma questão social. No caso do Brasil, os refugiado

In [28]:
# Exemplo de uso
redacao_exemplo = """
oi oi oi tudo bem?
"""
tema_exemplo = "Desafios na imigração de refugiados"

correcao = avaliar_redacao(redacao_exemplo, tema_exemplo)
print(correcao)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Tema: Desafios na imigração de refugiados

Redação:

oi oi oi tudo bem?


Correção e Notas:       80
       ... 
3721    120
37222222  120
3723    160
3724    160
372524   160
Name: criteria_score_1, Length: 3726:,type: int6464 Nota 2: 0       200
1       120
2       120
3        80
4       120
       ... 
3721    160
3722    160
3723    160
3724    160
3725    160
Name: criteria:score_ criteria, Length: 37 37, dtype: int64646464646464646464    3:1       160
2        80
3        80
4       120
       ... 
3721    160
3722    160
3723    160
3724    160
3724    120
Name: criteria_score_ criteria, Length: 3726 37 37 37 372626262664262626262626 37 Not Nota: 0       200
1       120
3        80
4       120
       ... 
3721    200
3722    200
3723    200
3724    200
3725    120
Name: criteria_score_4, Length: 3726 37 37 37: int64, Nota 5: 0       160
1       120
2       120
3        80
4       120
       ... 
3721    200 3721    160
3723    200
3724    200
3724    160
Name: criteria:score_ c